In [18]:
import os
from os import walk
import warnings
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import mlflow
import mlflow.sklearn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    confusion_matrix,
    classification_report,
    accuracy_score
)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import lightgbm
from lightgbm import LGBMRegressor

## Loading and processing data

In [2]:
df = pd.read_csv('project1_output.csv')

#### Converting cols to their appropriate types again because we lost it on the csv export

In [3]:
df["product_category_name"] = df["product_category_name"].astype('category')
df["order_status"] = df["order_status"].astype('category')
df["review_score"] = df["review_score"].astype('category')
df["payment_type"] = df["payment_type"].astype('category')
df["customer_zip_code_prefix"] = df["customer_zip_code_prefix"].astype('category')
df["customer_city"] = df["customer_city"].astype('category')
df["customer_state"] = df["customer_state"].astype('category')
df["seller_zip_code_prefix"] = df["seller_zip_code_prefix"].astype('category')
df["seller_city"] = df["seller_city"].astype('category')
df["seller_state"] = df["seller_state"].astype('category')

df["product_name_lenght"] = df["product_name_lenght"].astype('int64')
df["product_description_lenght"] = df["product_description_lenght"].astype('int64')
df["product_photos_qty"] = df["product_photos_qty"].astype('int64')
df["payment_installments"] = df["payment_installments"].astype('int64')
df["payment_sequential"] = df["payment_sequential"].astype('int64')

In [4]:
df.dtypes

order_id                           object
order_item_id                       int64
product_id                         object
seller_id                          object
shipping_limit_date                object
price                             float64
freight_value                     float64
product_category_name            category
product_name_lenght                 int64
product_description_lenght          int64
product_photos_qty                  int64
product_weight_g                  float64
product_length_cm                 float64
product_height_cm                 float64
product_width_cm                  float64
customer_id                        object
order_status                     category
order_purchase_timestamp           object
order_approved_at                  object
order_delivered_carrier_date       object
order_delivered_customer_date      object
order_estimated_delivery_date      object
review_id                          object
review_score                     c

#### Droping unnecessary columns
We only want to work with numerical values.

In [5]:
df = df.select_dtypes(exclude=['object'])

In [6]:
df.columns

Index(['order_item_id', 'price', 'freight_value', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'order_status', 'review_score',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'seller_zip_code_prefix', 'seller_city',
       'seller_state', 'payment_value_norm', 'volume'],
      dtype='object')

We'll also drop the `payment_value` column because our model would simply infer our target value from it by subtracting it from the `price` column.

In [7]:
df = df.drop(columns=['payment_value'])

### Feature engineering

Here we'll one-hot encode all of our categorical columns, and then drop the original ones

In [8]:
df = pd.get_dummies(df)
df = df.select_dtypes(exclude=['category'])

Even though we generated over 22000 columns this way, we believe that our model will be powerful enough to filter out any unecessary data.

## Picking column for prediction

We chose the `freight_value` column so we can perform a regression in order to try to find it's value based on all of the columns we have available.

In [9]:
TARGET_VALUE = 'freight_value'

In [10]:
target_col = df[TARGET_VALUE]

In [11]:
target_col

0         13.29
1         19.93
2         17.87
3         12.79
4         18.14
          ...  
118290    43.41
118291    36.53
118292    16.95
118293     8.72
118294    12.79
Name: freight_value, Length: 118295, dtype: float64

In [12]:
df = df.drop(columns=[TARGET_VALUE])

## Separating prediction and test data

We'll split our data in a 60/20/20 ratio.

In [13]:
# input 
train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

# output
train_labels, val_labels, test_labels = (
    np.split(
        target_col, 
        [int(.6*len(target_col)), int(.8*len(target_col))])
)

# Picking 4 ML algorithms

We'll use the following 4 algorithms:

1. Linear regression
2. Multilayer perceptron (a shallow one)
3. random forests
4. lightgbm/xgboost

### Metrics function

In [14]:
# Evaluate metrics
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

### Enabling MLFlow autologging

In [15]:
mlflow.sklearn.autolog()
mlflow.tensorflow.autolog()

2021/08/14 15:20:59 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


## Linear regression
Let's start off with linear regression, which is the most simple algorithm in our selection, and will serve as a baseline for the following algorithms.

In [20]:
def linear_regression(features, labels):
    train, val, test = features
    train_labels, val_labels, test_labels = labels
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Start an MLflow run; the "with" keyword ensures we'll close the run even if this cell crashes
    with mlflow.start_run():
        reg = LinearRegression()
        reg.fit(train, train_labels)

        predictions = reg.predict(val)

        (rmse, mae, r2) = eval_metrics(val_labels, predictions)

        # Print out ElasticNet model metrics
        print("Linear regression model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.sklearn.log_model(reg, "model")
        modelpath = "./mlflow/freight_value/model-linear-reg"
        mlflow.sklearn.save_model(lr, modelpath)

## Multilayer Perceptron

In [ ]:
def mlp(features, labels, params: dict):
    # hyper-parameters to test
    hidden_units = params["hidden_units"]
    lr = params["lr"]
    epochs = params["epochs"]
    
    train, val, test = features
    train_labels, val_labels, test_labels = labels
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    # Start an MLflow run; the "with" keyword ensures we'll close the run even if this cell crashes
    with mlflow.start_run():
        normalizer = preprocessing.Normalization(axis=-1)
        normalizer.adapt(np.array(train))
        
        mlp_model = tf.keras.Sequential([
            normalizer,
            layers.Dense(units=hidden_units)
            layers.Dense(units=hidden_units)
            layers.Dense(units=hidden_units)
            layers.Dense(units=1)
        ])

        mlp_model.summary()
        
        mlp_model.compile(
            optimizer=tf.optimizers.Adam(learning_rate=lr),
            loss='mean_absolute_error'
        )

        history = mlp_model.fit(
            train, train_labels,
            validation_data=(test, test_labels),
            epochs=epochs
        )
        
#         predictions = mlp_model.predict()

        (rmse, mae, r2) = eval_metrics(val_labels, predictions)

        # Print out ElasticNet model metrics
        print("Linear regression model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.sklearn.log_model(reg, "model")
        modelpath = "./mlflow/freight_value/model-linear-reg"
        mlflow.sklearn.save_model(lr, modelpath)

## Random Forest

In [31]:
def random_forest(features, labels, params: dict):
    train, val, test = features
    train_labels, val_labels, test_labels = labels
    
    warnings.filterwarnings("ignore")
    np.random.seed(40) # ?
    
    with mlflow.start_run():
        RandomForestRegressor(max_depth=3, random_state=0)
        # n_estimators: number of trees in the random forest
        rf.fit(train, train_labels)
        
        predictions = rf.predict(test)
        
        (rmse, mae, r2) = eval_metrics(val_labels, predictions)
        
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.sklearn.log_model(reg, "model")
        modelpath = "./mlflow/freight_value/model-linear-reg"
        mlflow.sklearn.save_model(lr, modelpath)

## Gradient Boosting with LightGBM

In [ ]:
def gradient_boosting(features, labels, params: dict):
    train, val, test = features
    train_labels, val_labels, test_labels = labels
    
    warnings.filterwarnings("ignore")
    np.random.seed(40) # ?
    
    with mlflow.start_run():
        model = LGBMRegressor()
        model.fit(train, test)
        
        predictions = model.predict(test)
        print('Prediction: %.3f' % predictions[0])
        
        (rmse, mae, r2) = eval_metrics(val_labels, predictions)

        # Print out ElasticNet model metrics
        print("Linear regression model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.sklearn.log_model(reg, "model")
        modelpath = "./mlflow/freight_value/model-linear-reg"
        mlflow.sklearn.save_model(lr, modelpath)